In [1]:
import os

try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args
    snakemake = load_rule_args(
        snakefile = os.getcwd() + '/../Snakefile',
        rule_name = 'liftover_glow',
        root=os.getcwd() + "/..",
    )

['Prostate', 'Breast_Mammary_Tissue', 'Adipose_Subcutaneous', 'Liver', 'Artery_Coronary', 'Cells_EBV_transformed_lymphocytes', 'Brain_Anterior_cingulate_cortex_BA24', 'Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Substantia_nigra', 'Artery_Aorta', 'Thyroid', 'Adrenal_Gland', 'Heart_Left_Ventricle', 'Spleen', 'Esophagus_Muscularis', 'Esophagus_Gastroesophageal_Junction', 'Brain_Caudate_basal_ganglia', 'Heart_Atrial_Appendage', 'Minor_Salivary_Gland', 'Lung', 'Nerve_Tibial', 'Whole_Blood', 'Testis', 'Brain_Cortex', 'Esophagus_Mucosa', 'Fibroblasts_Prokisch', 'Uterus', 'Muscle_Skeletal', 'Brain_Cerebellar_Hemisphere', 'Small_Intestine_Terminal_Ileum', 'Pancreas', 'Brain_Putamen_basal_ganglia', 'Brain_Spinal_cord_cervical_c_1', 'Pituitary', 'Skin_Not_Sun_Exposed_Suprapubic', 'Brain_Cerebellum', 'Artery_Tibial', 'Brain_Hypothalamus', 'Skin_Sun_Exposed_Lower_leg', 'Vagina', 'Brain_Amygdala', 'Ovary', 'Brain_Frontal_Cortex_BA9', 'Adipose_Visceral_Omentum', 'Colon_Sigmoid', 'Cells_Transforme

In [2]:
import os
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import glow
MEM = os.popen("ulimit -m").read()
if MEM.startswith("unlimited"):
    print("Memory not constrained, using all available memory...")
    import psutil
    MEM = psutil.virtual_memory().available / 1024
MEM = int(MEM)
N_CPU = int(os.popen("nproc").read())
print("memory: %dk" % MEM)
print("number of cores: %d" % N_CPU)
os.environ['PYSPARK_SUBMIT_ARGS'] = " ".join([
    '--driver-memory %dk' % MEM,
    'pyspark-shell'
])
os.environ['PYSPARK_SUBMIT_ARGS']
MAX_FAILURES=4
spark = (
    SparkSession.builder
    .appName('glow_liftover')
    .config("spark.jars.packages", ",".join([
        "io.projectglow:glow-spark3_2.12:1.0.0",
#         "io.delta:delta-core_2.12:0.8.0",
    ]))
#    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
#    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.local.dir", os.environ.get("TMP"))
    .config("spark.master", f"local[{N_CPU},{MAX_FAILURES}]")
    .config("spark.sql.shuffle.partitions", "2001")
#     .config("spark.sql.execution.arrow.enabled", "true")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
#     .config("spark.sql.execution.useObjectHashAggregateExec", "false")
    .config("spark.network.timeout", "1800s")
    .config("spark.driver.maxResultSize", "48G")
    .config("spark.files.maxPartitionBytes", 33554432)
#     .config("spark.databricks.io.cache.enabled", "true") # only enable when local storage is actually on local SSD
    .config("spark.task.maxFailures", MAX_FAILURES)
    .getOrCreate()
)
glow.register(spark)
spark

memory: 1024000000k
number of cores: 256


In [3]:
df = (
    spark
    .read
    .option("flattenInfoFields", False)
    .format('vcf')
    .load(snakemake.input['input_vcf'])
)

In [4]:
df = df.where(
    f.col('contigName').isin(snakemake.params['chroms'])
)

In [5]:
# sort by chromosome and by location
df = (
   df
   .repartitionByRange(4096, f.col("contigName"), f.col("start"))
   .sortWithinPartitions(["contigName", "start"])
   .persist()
)

In [6]:
# df.printSchema()

In [9]:
# df.rdd.getNumPartitions()

In [10]:
df = glow.transform("split_multiallelics", df)
df = glow.transform("normalize_variants", df, reference_genome_path=snakemake.input['reference_fasta'])

In [11]:
lifted_df = glow.transform('lift_over_variants', df, chain_file=snakemake.input['chain_file'], reference_file=snakemake.input['reference_fasta'])

In [15]:
# lifted_df.write.format("bigvcf").save(snakemake.output['lifted_vcf'])
lifted_df.write.format("vcf").save(snakemake.output['lifted_vcf'])

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/envs/florian3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/envs/florian3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/modules/i12g/anaconda/envs/florian3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o127.save